In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

url = 'https://mvic.sos.state.mi.us/Voter/GetMvicBallot/1828/679/'

response = requests.get(url,
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Geck o/20100101 Firefox/40.1'
            },
            verify=False,
)

response.status_code

/Users/Browning/Documents/citizenlabs/elections-api/.venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


200

In [3]:
html = response.text.strip()

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)

In [5]:
ballot = soup.find(id='PreviewMvicBallot').div.div.find_all('div', recursive=False)[1]

In [10]:
data = {}
section = None
division = None
office = None

offices = ballot.find(id='generalElectionOffices')

for index, item in enumerate(offices.find_all('div', {"class": ["section", "division", "office", "term", "candidate"]}), start=1):
    
    if "section" in item['class']:
        section = {}
        division = None
        office = None
        label = item.text
        data[label] = section
    
    elif "division" in item['class']:
        label = item.text.replace(' - Continued', '')
        
        try:
            division = section[label]
        except KeyError:
            division = {}
            
        section[label] = division
        office = None
        
    elif "office" in item['class']:
        label = item.text
        
        try:
            office = division[label]
        except KeyError:
            office = {'term': [], 'candidates': []}
            
        division[label] = office
        
    elif "term" in item['class']:
        office['term'].append(item.text)
        
    elif "candidate" in item['class']:
        office['candidates'].append(item.text)
        
    
from pprint import pprint
pprint(data)

{'NONPARTISAN SECTION': {'CITY': {'COMMISSIONER BY WARD': {'candidates': ['Allison '
                                                                          'Kay '
                                                                          'Lutz',
                                                                          'Jon '
                                                                          "O'Connor"],
                                                           'term': ['WARD 1',
                                                                    'Vote for '
                                                                    'not more '
                                                                    'than 1']},
                                  'LIBRARY BOARD DIRECTOR': {'candidates': ['Kathryn '
                                                                            'Dilley',
                                                                            'Jonathan '
     

In [11]:
data = {}
section = None
division = None
office = None

proposals = ballot.find(id='proposals')

for index, item in enumerate(proposals.find_all('div', {"class": ["section", "division", "proposalTitle", "proposalText"]}), start=1):
    
    if "section" in item['class']:
        section = {}
        division = None
        office = None
        label = item.text
        data[label] = section
        
    elif "division" in item['class']:
        label = item.text.replace(' - Continued', '')
        
        try:
            division = section[label]
        except KeyError:
            division = {}
            
        section[label] = division
        office = None
        
    elif "proposalTitle" in item['class']:
        label = item.text
        
        try:
            proposal = division[label]
        except KeyError:
            proposal = {'text': None}
            
        division[label] = proposal
        
    elif "proposalText" in item['class']:
        proposal['text'] = item.text
        

from pprint import pprint
pprint(data)

{'PROPOSAL SECTION': {'CITY PROPOSALS': {'City of Grand Rapids\nCity of Grand Rapids Charter Amendment authorizing the levy of a permanent tax millage of 1.25 mills for PARKS, POOLS AND PLAYGROUNDS commencing January 1, 2021. The current levy of 0.98 mills expires in 2020.': {'text': '\xa0\n'
                                                                                                                                                                                                                                                                                             '\xa0\n'
                                                                                                                                                                                                                                                                                             '\xa0\n'
                                                                                                                      